In [2]:
print("qwerty")

qwerty


In [3]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
from langchain_pinecone import PineconeVectorStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.document_loaders import CSVLoader

c:\Users\ayush\.conda\envs\chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
import os

In [5]:
os.environ['PINECONE_API_KEY'] = "5c8d0e8e-4a48-4b6f-953f-e8cfad1c3f4e"
os.environ['PINECONE_INDEX_NAME'] = "chatbot-rag"


In [6]:
from pathlib import Path

In [7]:
PINECONE_API_KEY = "5c8d0e8e-4a48-4b6f-953f-e8cfad1c3f4e"
PINECONE_INDEX = "chatbot-rag"
INDEX_DIMENSIONS = 384

In [8]:
filepath = Path("data\data.csv")

In [9]:
loader = CSVLoader(file_path=filepath)

In [10]:
data = loader.load()

In [197]:
data[0]

Document(page_content='Year: 1981\nCountry Name: Brazil\nIndicator Name: Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)\nIndicator Code: SI.POV.DDAY\nSOURCE_NOTE: Poverty headcount ratio at $2.15 a day is the percentage of the population living on less than $2.15 a day at 2017 purchasing power adjusted prices. As a result of revisions in PPP exchange rates, poverty rates for individual countries cannot be compared with poverty rates reported in earlier editions.\nSOURCE_ORGANIZATION: World Bank, Poverty and Inequality Platform. Data are based on primary household survey data obtained from government statistical agencies and World Bank country departments. Data for high-income economies are mostly from the Luxembourg Income Study database. For more information and methodology, please see http://pip.worldbank.org.\nRegion: Latin America & Caribbean\nIncomeGroup: Upper middle income\nValue: 23.9', metadata={'source': 'data\\data.csv', 'row': 0})

In [11]:
len(data)

104

In [12]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 90)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [13]:
text_chunks = text_split(data)
print("Number of chunks:", len(text_chunks))

Number of chunks: 104


In [14]:
text_chunks[1]

Document(page_content='Year: 1982\nCountry Name: Brazil\nIndicator Name: Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)\nIndicator Code: SI.POV.DDAY\nSOURCE_NOTE: Poverty headcount ratio at $2.15 a day is the percentage of the population living on less than $2.15 a day at 2017 purchasing power adjusted prices. As a result of revisions in PPP exchange rates, poverty rates for individual countries cannot be compared with poverty rates reported in earlier editions.\nSOURCE_ORGANIZATION: World Bank, Poverty and Inequality Platform. Data are based on primary household survey data obtained from government statistical agencies and World Bank country departments. Data for high-income economies are mostly from the Luxembourg Income Study database. For more information and methodology, please see http://pip.worldbank.org.\nRegion: Latin America & Caribbean\nIncomeGroup: Upper middle income\nValue: 24.4', metadata={'source': 'data\\data.csv', 'row': 1})

In [15]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

In [17]:
query_result = embeddings.embed_query("Sample")
print("Length", len(query_result))

Length 384


In [18]:
index_name= PINECONE_INDEX

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], index_name=index_name,embedding=embeddings)

In [198]:

query = "People without jobs in 2010"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Year: 2010\nCountry Name: Brazil\nIndicator Name: Unemployment, total (% of total labor force) (modeled ILO estimate)\nIndicator Code: SL.UEM.TOTL.ZS\nSOURCE_NOTE: Unemployment refers to the share of the labor force that is without work but available for and seeking employment.\nSOURCE_ORGANIZATION: International Labour Organization. “ILO Modelled Estimates and Projections database (ILOEST)” ILOSTAT. Accessed February 06, 2024. https://ilostat.ilo.org/data/.\nRegion: Latin America & Caribbean\nIncomeGroup: Upper middle income\nValue: 8.426'), Document(page_content='Year: 2010\nCountry Name: Brazil\nIndicator Name: Unemployment, total (% of total labor force) (modeled ILO estimate)\nIndicator Code: SL.UEM.TOTL.ZS\nSOURCE_NOTE: Unemployment refers to the share of the labor force that is without work but available for and seeking employment.\nSOURCE_ORGANIZATION: International Labour Organization. “ILO Modelled Estimates and Projections database (ILOEST)” IL

In [177]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [178]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [179]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.3})

In [180]:
# from langchain.schema.retriever import BaseRetriever, Document
# from typing import List

# class SimpleRetriever(BaseRetriever):
#     def __init__(self, vector_store, k=5):
#         self.vector_store = vector_store
#         self.k = k

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         """Retrieve documents based on embeddings similarity, limited to the top k results."""
#         # Assuming `vector_store` has a method `similarity_search` which takes a query and returns documents
#         results = self.vector_store.similarity_search(query, k=self.k)
#         return results

#     async def _aget_relevant_documents(self, query: str) -> List[Document]:
#         """Asynchronous retrieval of documents."""
#         return await self.vector_store.similarity_search(query, k=self.k)


In [181]:
from typing import List, Optional, Any  # Adding 'Any' to the import
from langchain.schema import Document, BaseRetriever
from pydantic import BaseModel, Field

In [182]:
# from pydantic import BaseModel, Field
# from typing import List, Optional
# from langchain.schema import Document, BaseRetriever

# class SimpleRetriever(BaseRetriever):
#     vector_store: Any  # Specify the correct type if known, else use `Any`
#     k: int = Field(default=5, description="The number of documents to retrieve")

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         """Retrieve documents based on embeddings similarity, limited to the top k results."""
#         # Simulate fetching documents based on a query
#         # This is where you'd normally interact with the vector store
#         return self.vector_store.get_relevant_documents(query, k=self.k)

#     async def _aget_relevant_documents(self, query: str) -> List[Document]:
#         """Asynchronously retrieve documents."""
#         return await self.vector_store.get_relevant_documents(query, k=self.k)


In [183]:
from langchain.schema import Document, BaseRetriever
from typing import List, Any
from pydantic import BaseModel, Field

class SimpleRetriever(BaseRetriever):
    vector_store: Any  # Specify the correct type if known, else use `Any`
    k: int = Field(default=5, description="The number of documents to retrieve")

    def _get_relevant_documents(self, query: str) -> List[Document]:
        """Retrieve documents based on embeddings similarity, limited to the top k results using cosine similarity."""
        return self.vector_store.similarity_search(query=query, k=self.k)

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        """Asynchronously retrieve documents using cosine similarity."""
        return await self.vector_store.similarity_search(query=query, k=self.k)


In [184]:
# from langchain.schema import Document, BaseRetriever
# from typing import List, Any
# from pydantic import BaseModel, Field

# class SimpleRetriever(BaseRetriever):
#     vector_store: Any  # Specify the correct type if known, else use `Any`
#     k: int = Field(default=5, description="The number of documents to retrieve")

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         """Retrieve documents based on embeddings similarity, limited to the top k results using cosine similarity."""
#         return self.vector_store.search(query=query, k=self.k, search_type='similarity')

#     async def _aget_relevant_documents(self, query: str) -> List[Document]:
#         """Asynchronously retrieve documents using cosine similarity."""
#         return await self.vector_store.search(query=query, k=self.k, search_type='similarity')


In [185]:
# Assuming docsearch is already set up as an instance of PineconeVectorStore
simple_retriever = SimpleRetriever(vector_store=docsearch, k=3)


In [193]:
doc = simple_retriever._get_relevant_documents("number of internet users in 2012")

In [194]:
doc

[Document(page_content='Year: 2012\nCountry Name: Brazil\nIndicator Name: Individuals using the Internet (% of population)\nIndicator Code: IT.NET.USER.ZS\nSOURCE_NOTE: Internet users are individuals who have used the Internet (from any location) in the last 3 months. The Internet can be used via a computer, mobile phone, personal digital assistant, games machine, digital TV etc.\nSOURCE_ORGANIZATION: International Telecommunication Union (ITU) World Telecommunication/ICT Indicators Database\nRegion: Latin America & Caribbean\nIncomeGroup: Upper middle income\nValue: 48.56'),
 Document(page_content='Year: 2012\nCountry Name: Brazil\nIndicator Name: Individuals using the Internet (% of population)\nIndicator Code: IT.NET.USER.ZS\nSOURCE_NOTE: Internet users are individuals who have used the Internet (from any location) in the last 3 months. The Internet can be used via a computer, mobile phone, personal digital assistant, games machine, digital TV etc.\nSOURCE_ORGANIZATION: Internationa

In [199]:
qa = RetrievalQA.from_chain_type(  
    llm=llm,  
    chain_type="stuff",  
    retriever=simple_retriever,
    #chain_type_kwargs=chain_type_kwargs 
)  

In [200]:

user_input=input(f"Input Prompt:")


In [201]:
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :   What is: No: Yes | Asked Questions: What percentage of the following the following the following the following the following the question: Brazil's: What is:
What was provided by Country: What is: What is: Does anyone knows the following the following the following the following the following the following the following the following the following the following guidelic:
What is: No.
